In [26]:
import requests
import psycopg2
import pandas as pd
from pandas.io.json import json_normalize

def get_data():
	r = requests.get('http://www.citibikenyc.com/stations/json')
	return(r)

def setup_db():
	# Connect to an existing database
	conn = psycopg2.connect("dbname=getting_started user=postgres password=Gatsby")

	with conn:
	    #open the cursor
	    cur = conn.cursor()

	    # set up the tables.
	    cur.execute("DROP TABLE IF EXISTS citibike_reference")

	    # CHANGED TO TEXT TYPE TO ELIMINATE INPUT ERRORS

	    cur.execute("CREATE TABLE citibike_reference (id integer PRIMARY KEY, \
	    	totalDocks integer NOT NULL DEFAULT 0, \
	    	city TEXT, \
		    altitude TEXT, \
		    stAddress2 TEXT, \
		    longitude NUMERIC NOT NULL DEFAULT 0, \
		    postalCode TEXT, \
		    testStation TEXT, \
		    stAddress1 TEXT, \
		    stationName TEXT, \
		    landMark TEXT, \
		    latitude NUMERIC NOT NULL DEFAULT 0, \
		    location TEXT);")
	return

def load_db(r):

	conn = psycopg2.connect("dbname=getting_started user=postgres password=Gatsby")

	
	#a prepared SQL statement we're going to execute over and over again
	sql = "INSERT INTO citibike_reference (id, totalDocks, city, altitude, stAddress2, \
		longitude, postalCode, testStation, stAddress1, stationName, landMark, latitude, location) \
		VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

# 	sql = "INSERT INTO citibike_reference VALUES (%(id)s, %(totalDocks)s, %(city)s, %(altitude)s, %(stAddress2)s,\
# 												%(longitude)s, %(postalCode)s, %(testStation)s, %(stAddress1)s,\
# 												%(stationName)s, %(landMark)s, %(latitude)s, %(location)s)"

# cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)",
# ...      (100, "abc'def"))

	#for loop to populate values in the database
	with conn:
		cur = conn.cursor()
		for station in r.json()['stationBeanList']:

			#####  ***** Need to do some bounds checking here ***** ######
			cur.execute(sql, (station['id'],station['totalDocks'],station['city'],station['altitude'], station['stAddress2'], station['longitude'],station['postalCode'],station['testStation'], station['stAddress1'],station['stationName'],station['landMark'],station['latitude'],station['location']))

	return

def fix_ids(r):
	conn = psycopg2.connect("dbname=getting_started user=postgres password=Gatsby")
	#extract the column from the DataFrame and put them into a list
	#from pandas.io.json import json_normalize

	df = json_normalize(r.json()['stationBeanList'])
	station_ids = df['id'].tolist() 

	#add the '_' to the station name and also add the data type for SQLite
	station_ids = ['_' + str(x) + ' INT' for x in station_ids]
# 	station_ids = ['_' + str(x) for x in station_ids] # removing int here
	print("************ printing station ids")
	print(station_ids)

	#create the table
	#in this case, we're concatenating the string and joining all the station ids (now with '_' and 'INT' added)
	with conn:
		cur = conn.cursor()
		cur.execute("DROP TABLE IF EXISTS available_bikes")
		# cur.execute("CREATE TABLE available_bikes (execution_time INT, " +  ", ".join(station_ids) + ");")
		# cur.execute("CREATE TABLE available_bikes execution_time INT, station_ids TEXT")
		cur.execute("CREATE TABLE available_bikes (execution_time integer PRIMARY KEY, " + ", ".join(station_ids) + ")")

		# cur.execute("CREATE TABLE available_bikes (id integer PRIMARY KEY, \
	 #    	totalDocks integer NOT NULL DEFAULT 0, \
	 #    	city TEXT, \
		#     altitude TEXT, \
		# This is creating a table with a column for every execution time + stating ID pair?  Why?
		# is this vulnerable to sql injection if data contained right string in station_ids???
	return

def fix_dates(r):
	conn = psycopg2.connect("dbname=getting_started user=postgres password=Gatsby")
	 #a package with datetime objects
	import time

	# a package for parsing a string into a Python datetime object
	from dateutil.parser import parse 

	import collections
	import datetime
	# 	 #take the string and parse it into a Python datetime object
	exec_time = parse(r.json()['executionTime'])
# 	new_time = exec_time.strftime('%Y-%m-%dT%H:%M:%S')
	s =(exec_time - datetime.datetime(1970,1,1)).total_seconds()
	print(s)

	with conn:
		cur = conn.cursor()
		# cur.execute('INSERT INTO available_bikes (execution_time) VALUES(%s)', ('new_time'))
		cur.execute('INSERT INTO available_bikes (execution_time) VALUES(%s)', (s,))
		##***********  getting error in line above

	# cur.executemany("INSERT INTO weather2 VALUES(%(city)s, %(year)s, %(warm_month)s, %(cold_month)s)", weather)



In [37]:
r.json().keys()


[u'executionTime', u'stationBeanList']

In [38]:
r.json()['stationBeanList']

[{u'altitude': u'',
  u'availableBikes': 1,
  u'availableDocks': 38,
  u'city': u'',
  u'id': 72,
  u'landMark': u'',
  u'lastCommunicationTime': u'2016-04-07 10:18:08 AM',
  u'latitude': 40.76727216,
  u'location': u'',
  u'longitude': -73.99392888,
  u'postalCode': u'',
  u'stAddress1': u'W 52 St & 11 Ave',
  u'stAddress2': u'',
  u'stationName': u'W 52 St & 11 Ave',
  u'statusKey': 1,
  u'statusValue': u'In Service',
  u'testStation': False,
  u'totalDocks': 39},
 {u'altitude': u'',
  u'availableBikes': 20,
  u'availableDocks': 13,
  u'city': u'',
  u'id': 79,
  u'landMark': u'',
  u'lastCommunicationTime': u'2016-04-07 10:15:56 AM',
  u'latitude': 40.71911552,
  u'location': u'',
  u'longitude': -74.00666661,
  u'postalCode': u'',
  u'stAddress1': u'Franklin St & W Broadway',
  u'stAddress2': u'',
  u'stationName': u'Franklin St & W Broadway',
  u'statusKey': 1,
  u'statusValue': u'In Service',
  u'testStation': False,
  u'totalDocks': 33},
 {u'altitude': u'',
  u'availableBikes': 

In [39]:
#!/usr/bin/python
import time

print "Start : %s" % time.ctime()
time.sleep( 5 )
print "End : %s" % time.ctime()

Start : Wed Apr 13 20:39:45 2016
End : Wed Apr 13 20:39:50 2016


In [35]:
import collections

# Then we iterate through the stations in the "stationBeanList":  **************** rest of this function untested ****************

id_bikes = collections.defaultdict(int) #defaultdict to store available bikes by station

#loop through the stations in the station list
for station in r.json()['stationBeanList']:
    id_bikes[station['id']] = station['availableBikes']

In [36]:
id_bikes

defaultdict(int,
            {72: 1,
             79: 20,
             82: 6,
             83: 26,
             116: 4,
             119: 11,
             120: 4,
             127: 0,
             128: 30,
             137: 46,
             143: 1,
             144: 2,
             146: 4,
             147: 18,
             150: 0,
             151: 2,
             152: 1,
             153: 20,
             157: 0,
             160: 0,
             161: 5,
             164: 38,
             167: 12,
             168: 30,
             173: 17,
             174: 0,
             195: 3,
             212: 21,
             216: 7,
             217: 21,
             223: 0,
             224: 10,
             225: 18,
             228: 53,
             229: 20,
             232: 4,
             233: 16,
             236: 0,
             237: 0,
             238: 1,
             239: 29,
             241: 0,
             242: 13,
             243: 15,
             244: 0,
             245: 5,


In [ ]:
r.

In [30]:
r.

AttributeError: 'Response' object has no attribute 'keys'

In [4]:
r = get_data()
conn = psycopg2.connect("dbname=getting_started user=postgres password=Gatsby")
#extract the column from the DataFrame and put them into a list
#from pandas.io.json import json_normalize

df = json_normalize(r.json()['stationBeanList'])
station_ids = df['id'].tolist() 

#add the '_' to the station name and also add the data type for SQLite
# station_ids = ['_' + str(x) + ' INT' for x in station_ids]
station_ids = ['_' + str(x) for x in station_ids] # removing int here
print("************ printing station ids")
print(station_ids)

************ printing station ids
['_72', '_79', '_82', '_83', '_116', '_119', '_120', '_127', '_128', '_137', '_143', '_144', '_146', '_147', '_150', '_151', '_152', '_153', '_157', '_160', '_161', '_164', '_167', '_168', '_173', '_174', '_195', '_212', '_216', '_217', '_223', '_224', '_225', '_228', '_229', '_232', '_233', '_236', '_237', '_238', '_239', '_241', '_242', '_243', '_244', '_245', '_247', '_248', '_249', '_250', '_251', '_252', '_253', '_254', '_257', '_258', '_259', '_260', '_261', '_262', '_263', '_264', '_265', '_266', '_267', '_268', '_270', '_274', '_275', '_276', '_278', '_279', '_280', '_281', '_282', '_284', '_285', '_289', '_291', '_293', '_295', '_296', '_297', '_298', '_300', '_301', '_302', '_303', '_304', '_305', '_306', '_307', '_308', '_309', '_310', '_311', '_312', '_313', '_315', '_316', '_317', '_319', '_320', '_321', '_322', '_323', '_324', '_325', '_326', '_327', '_328', '_329', '_330', '_331', '_332', '_334', '_335', '_336', '_337', '_339', '_340', '

In [5]:
# cur.execute("CREATE TABLE available_bikes (execution_time integer PRIMARY KEY, " + ", ".join(station_ids) + ")")
print(",".join(station_ids))

_72,_79,_82,_83,_116,_119,_120,_127,_128,_137,_143,_144,_146,_147,_150,_151,_152,_153,_157,_160,_161,_164,_167,_168,_173,_174,_195,_212,_216,_217,_223,_224,_225,_228,_229,_232,_233,_236,_237,_238,_239,_241,_242,_243,_244,_245,_247,_248,_249,_250,_251,_252,_253,_254,_257,_258,_259,_260,_261,_262,_263,_264,_265,_266,_267,_268,_270,_274,_275,_276,_278,_279,_280,_281,_282,_284,_285,_289,_291,_293,_295,_296,_297,_298,_300,_301,_302,_303,_304,_305,_306,_307,_308,_309,_310,_311,_312,_313,_315,_316,_317,_319,_320,_321,_322,_323,_324,_325,_326,_327,_328,_329,_330,_331,_332,_334,_335,_336,_337,_339,_340,_341,_342,_343,_344,_345,_346,_347,_348,_349,_350,_351,_352,_353,_354,_355,_356,_357,_358,_359,_360,_361,_362,_363,_364,_365,_366,_367,_368,_369,_372,_373,_376,_377,_379,_380,_382,_383,_384,_385,_386,_387,_388,_389,_390,_391,_392,_393,_394,_395,_396,_397,_398,_399,_400,_401,_402,_403,_405,_406,_407,_408,_409,_410,_411,_412,_414,_415,_416,_417,_418,_419,_420,_421,_422,_423,_426,_427,_428,_430,_432

In [6]:
main()


NameError: name 'main' is not defined

In [7]:
def prep_db():
	r = get_data()
	setup_db()
	load_db(r)
	fix_ids(r)
	fix_dates(r)

	return(True)
  
def main():
    prep_db()

In [8]:
setup_db()

In [9]:
load_db(r)

In [12]:
fix_ids(r)

************ printing station ids
['_72 INT', '_79 INT', '_82 INT', '_83 INT', '_116 INT', '_119 INT', '_120 INT', '_127 INT', '_128 INT', '_137 INT', '_143 INT', '_144 INT', '_146 INT', '_147 INT', '_150 INT', '_151 INT', '_152 INT', '_153 INT', '_157 INT', '_160 INT', '_161 INT', '_164 INT', '_167 INT', '_168 INT', '_173 INT', '_174 INT', '_195 INT', '_212 INT', '_216 INT', '_217 INT', '_223 INT', '_224 INT', '_225 INT', '_228 INT', '_229 INT', '_232 INT', '_233 INT', '_236 INT', '_237 INT', '_238 INT', '_239 INT', '_241 INT', '_242 INT', '_243 INT', '_244 INT', '_245 INT', '_247 INT', '_248 INT', '_249 INT', '_250 INT', '_251 INT', '_252 INT', '_253 INT', '_254 INT', '_257 INT', '_258 INT', '_259 INT', '_260 INT', '_261 INT', '_262 INT', '_263 INT', '_264 INT', '_265 INT', '_266 INT', '_267 INT', '_268 INT', '_270 INT', '_274 INT', '_275 INT', '_276 INT', '_278 INT', '_279 INT', '_280 INT', '_281 INT', '_282 INT', '_284 INT', '_285 INT', '_289 INT', '_291 INT', '_293 INT', '_295 INT

In [15]:
fix_dates(r)

2016-04-07T10:18:22


TypeError: not all arguments converted during string formatting

In [ ]:
def prep_db():
	r = get_data()
	setup_db()
	load_db(r)
	fix_ids(r)
	fix_dates(r)

	return(True)
